### Customer Analysis SQL Queries

Demographic Distribution
:Analyze the distribution of customers based on gender, age, and location



In [1]:
import pandas as pd
import mysql.connector

In [2]:
connection=mysql.connector.connect(
    host='localhost',
    user='root',
    password='pochi2002*',
    database='Dataspark'
)
cursor=connection.cursor()
cursor

In [3]:
query='''SELECT 
    Gender, 
    YEAR(CURDATE()) - YEAR(Birthday) AS Age, 
    City, 
    State, 
    Country, 
    Continent,
    COUNT(CustomerKey) AS TotalCustomers
FROM customer
GROUP BY Gender, YEAR(CURDATE()) - YEAR(Birthday), City, State, Country, Continent
ORDER BY TotalCustomers DESC
'''
cursor.execute(query)

In [4]:
results = cursor.fetchall()
df = pd.DataFrame(results, columns=['Gender', 'Age', 'City', 'State', 'Country', 'Continent', 'TotalCustomers'])

df.head()


,Gender,Age,City,State,Country,Continent,TotalCustomers
0,Female,57,Toronto,Ontario,Canada,North America,6
1,Male,67,Toronto,Ontario,Canada,North America,5
2,Female,42,Toronto,Ontario,Canada,North America,5
3,Female,60,Toronto,Ontario,Canada,North America,5
4,Female,59,New York,New York,United States,North America,5


Purchase Patterns: Average order value, frequency of purchases, preferred products

In [5]:
query='''SELECT 
    c.CustomerKey, 
    c.Gender,
    COUNT(s.OrderNumber) AS FrequencyOfPurchases,
    AVG(s.Quantity * p.UnitPriceUSD) AS AverageOrderValue,
    p.ProductName AS PreferredProduct
FROM sales s
JOIN customer c ON s.CustomerKey = c.CustomerKey
JOIN product p ON s.ProductKey = p.ProductKey
GROUP BY c.CustomerKey, c.Gender, p.ProductName
ORDER BY FrequencyOfPurchases DESC, AverageOrderValue DESC'''
cursor.execute(query)


In [6]:
results = cursor.fetchall()
# Creating a DataFrame
df_purchase_patterns = pd.DataFrame(results, columns=['CustomerKey', 'Gender', 'FrequencyOfPurchases', 'AverageOrderValue', 'PreferredProduct'])
# Display the first few rows of the DataFrame
df_purchase_patterns.head()


,CustomerKey,Gender,FrequencyOfPurchases,AverageOrderValue,PreferredProduct
0,1948852,Male,3,3230.000000,Adventure Works Desktop PC2.33 XD233 Black
1,1253996,Female,3,1318.333333,Contoso Water Heater 4.0GPM M1250 Silver
2,1884663,Female,2,20799.935000,Litware Refrigerator 24.7CuFt X980 Green
3,433811,Male,2,7375.000000,Contoso Water Heater 7.2GPM X1800 Silver
4,1579183,Male,2,5329.500000,Adventure Works Desktop PC2.33 XD233 Black


Customer Segmentation: Segment customers based on age and frequency of purchases

In [7]:
query='''SELECT 
    c.CustomerKey, 
    c.Gender,
    CASE 
        WHEN YEAR(CURDATE()) - YEAR(c.Birthday) < 30 THEN 'Young'
        WHEN YEAR(CURDATE()) - YEAR(c.Birthday) BETWEEN 30 AND 50 THEN 'Middle-aged'
        ELSE 'Senior'
    END AS AgeGroup,
    COUNT(s.OrderNumber) AS FrequencyOfPurchases,
    AVG(s.Quantity * p.UnitPriceUSD) AS AverageOrderValue,
    CASE 
        WHEN COUNT(s.OrderNumber) >= 5 THEN 'High Frequency'
        WHEN COUNT(s.OrderNumber) BETWEEN 2 AND 4 THEN 'Medium Frequency'
        ELSE 'Low Frequency'
    END AS PurchaseFrequencySegment
FROM sales s
JOIN customer c ON s.CustomerKey = c.CustomerKey
JOIN product p ON s.ProductKey = p.ProductKey
GROUP BY c.CustomerKey, c.Gender, AgeGroup
ORDER BY FrequencyOfPurchases DESC'''
cursor.execute(query)

In [8]:
results = cursor.fetchall()
# Creating a DataFrame
df_segmentation = pd.DataFrame(results, columns=['CustomerKey', 'Gender', 'AgeGroup', 'FrequencyOfPurchases', 'AverageOrderValue', 'PurchaseFrequencySegment'])
# Display the first few rows of the DataFrame
df_segmentation.head()


,CustomerKey,Gender,AgeGroup,FrequencyOfPurchases,AverageOrderValue,PurchaseFrequencySegment
0,723572,Male,Middle-aged,36,1117.361389,High Frequency
1,1925694,Male,Senior,32,823.105938,High Frequency
2,1579183,Male,Middle-aged,32,1118.135313,High Frequency
3,1972249,Female,Middle-aged,29,995.498276,High Frequency
4,1942072,Male,Senior,29,864.800345,High Frequency


Top-spending customers based on total spend

In [9]:
query='''
SELECT 
    c.CustomerKey, 
    c.Gender,
    SUM(s.Quantity * p.UnitPriceUSD) AS TotalSpend,
    COUNT(s.OrderNumber) AS FrequencyOfPurchases
FROM sales s
JOIN customer c ON s.CustomerKey = c.CustomerKey
JOIN product p ON s.ProductKey = p.ProductKey
GROUP BY c.CustomerKey, c.Gender
ORDER BY TotalSpend DESC'''
cursor.execute(query)

In [10]:
results = cursor.fetchall()
# Creating a DataFrame
df_top_spending = pd.DataFrame(results, columns=['CustomerKey', 'Gender', 'TotalSpend', 'FrequencyOfPurchases'])
# Display the first few rows of the DataFrame
df_top_spending.head()

,CustomerKey,Gender,TotalSpend,FrequencyOfPurchases
0,1702221,Male,61871.70,21
1,1884663,Female,43517.80,9
2,1969704,Female,42788.04,18
3,535496,Female,41521.53,10
4,551036,Male,40556.54,10


### Product Analysis SQL Queries

Product Popularity: Identify the most and least popular products based on sales

In [11]:
cursor.execute("""
    SELECT 
        p.ProductName,
        SUM(s.Quantity) AS TotalQuantitySold,
        SUM(s.Quantity * p.UnitPriceUSD) AS TotalRevenue
    FROM sales s
    JOIN product p ON s.ProductKey = p.ProductKey
    GROUP BY p.ProductName
    ORDER BY TotalQuantitySold DESC;
""")

# Fetching the results
results = cursor.fetchall()

# Creating a DataFrame
df_product_popularity = pd.DataFrame(results, columns=['ProductName', 'TotalQuantitySold', 'TotalRevenue'])

# Display the first few rows of the DataFrame
df_product_popularity.head()


,ProductName,TotalQuantitySold,TotalRevenue
0,WWI Desktop PC2.33 X2330 Black,550,505450.00
1,WWI Desktop PC1.80 E1800 White,538,123686.20
2,Adventure Works Desktop PC1.60 ED160 Black,521,140643.95
3,Adventure Works Desktop PC2.30 MD230 White,521,312079.00
4,Adventure Works Desktop PC1.80 ED180 Black,520,191880.00


Profitability Analysis: Calculate profit margins for products

In [12]:
cursor.execute("""
    SELECT 
        p.ProductName,
        p.UnitCostUSD,
        p.UnitPriceUSD,
        (p.UnitPriceUSD - p.UnitCostUSD) AS ProfitMargin,
        ((p.UnitPriceUSD - p.UnitCostUSD) / p.UnitPriceUSD) * 100 AS ProfitMarginPercentage
    FROM product p
    ORDER BY ProfitMargin DESC;
""")

# Fetching the results
results = cursor.fetchall()

# Creating a DataFrame
df_profitability = pd.DataFrame(results, columns=['ProductName', 'UnitCostUSD', 'UnitPriceUSD', 'ProfitMargin', 'ProfitMarginPercentage'])

# Display the first few rows of the DataFrame
df_profitability.head()


,ProductName,UnitCostUSD,UnitPriceUSD,ProfitMargin,ProfitMarginPercentage
0,Fabrikam Refrigerator 24.7CuFt X9800 White,1060.22,3199.99,2139.77,66.868021
1,Fabrikam Refrigerator 24.7CuFt X9800 Brown,1060.22,3199.99,2139.77,66.868021
2,Fabrikam Refrigerator 24.7CuFt X9800 Silver,1060.22,3199.99,2139.77,66.868021
3,Fabrikam Refrigerator 24.7CuFt X9800 Green,1060.22,3199.99,2139.77,66.868021
4,Fabrikam Refrigerator 24.7CuFt X9800 Blue,1060.22,3199.99,2139.77,66.868021


Category Analysis: Analyze sales performance across different product categories and subcategories.


In [13]:
cursor.execute("""
    SELECT 
        p.Category,
        p.Subcategory,
        SUM(s.Quantity) AS TotalQuantitySold,
        SUM(s.Quantity * p.UnitPriceUSD) AS TotalRevenue
    FROM sales s
    JOIN product p ON s.ProductKey = p.ProductKey
    GROUP BY p.Category, p.Subcategory
    ORDER BY TotalRevenue DESC;
""")

# Fetching the results
results = cursor.fetchall()

# Creating a DataFrame
df_category_analysis = pd.DataFrame(results, columns=['Category', 'Subcategory', 'TotalQuantitySold', 'TotalRevenue'])

# Display the first few rows of the DataFrame
df_category_analysis.head()


,Category,Subcategory,TotalQuantitySold,TotalRevenue
0,Computers,Desktops,20626,9906356.50
1,TV and Video,Televisions,5625,4308719.19
2,Computers,Projectors & Screens,4757,3767522.00
3,Home Appliances,Water Heaters,4563,3547822.50
4,Cameras and camcorders,Camcorders,4482,3357990.00


### Sales Analysis SQL Queries

Overall Sales Performance: Total sales over time

In [14]:
query='''SELECT 
    YEAR(OrderDate) AS Year,
    MONTH(OrderDate) AS Month,
    SUM(s.Quantity * p.UnitPriceUSD) AS TotalSales
FROM sales s
JOIN product p ON s.ProductKey = p.ProductKey
GROUP BY YEAR(OrderDate), MONTH(OrderDate)
ORDER BY Year, Month'''
cursor.execute(query)
results = cursor.fetchall()
# Creating a DataFrame
df_sales_performance = pd.DataFrame(results, columns=['Year', 'Month', 'TotalSales'])
# Display the first few rows of the DataFrame
df_sales_performance.head()


,Year,Month,TotalSales
0,2016,1,649918.78
1,2016,2,891098.30
2,2016,3,338407.36
3,2016,4,110591.63
4,2016,5,595986.18


Sales by Product: Top performers in terms of quantity sold and revenue

In [15]:
cursor.execute("""
    SELECT 
        p.ProductName,
        SUM(s.Quantity) AS TotalQuantitySold,
        SUM(s.Quantity * p.UnitPriceUSD) AS TotalRevenue
    FROM sales s
    JOIN product p ON s.ProductKey = p.ProductKey
    GROUP BY p.ProductName
    ORDER BY TotalRevenue DESC;
""")
# Fetching the results
results = cursor.fetchall()
# Creating a DataFrame
df_sales_by_product = pd.DataFrame(results, columns=['ProductName', 'TotalQuantitySold', 'TotalRevenue'])
# Display the first few rows of the DataFrame
df_sales_by_product.head()

,ProductName,TotalQuantitySold,TotalRevenue
0,WWI Desktop PC2.33 X2330 Black,550,505450.00
1,Adventure Works Desktop PC2.33 XD233 Silver,481,466089.00
2,Adventure Works Desktop PC2.33 XD233 Brown,479,464151.00
3,Adventure Works Desktop PC2.33 XD233 Black,462,447678.00
4,Adventure Works Desktop PC2.33 XD233 White,451,437019.00


Sales by Store: Store performance analysis

In [16]:
cursor.execute("""
    SELECT 
        s.StoreKey,
        COUNT(s.OrderNumber) AS TotalOrders,
        SUM(s.Quantity * p.UnitPriceUSD) AS TotalSales
    FROM sales s
    JOIN product p ON s.ProductKey = p.ProductKey
    GROUP BY s.StoreKey
    ORDER BY TotalSales DESC;
""")

# Fetching the results
results = cursor.fetchall()

# Creating a DataFrame
df_sales_by_store = pd.DataFrame(results, columns=['StoreKey', 'TotalOrders', 'TotalSales'])

# Display the first few rows of the DataFrame
df_sales_by_store.head()


,StoreKey,TotalOrders,TotalSales
0,0,13165,11404324.63
1,55,1518,1417885.41
2,50,1519,1394738.06
3,54,1498,1384396.24
4,9,1577,1336150.06



Sales by Currency: Impact of different currencies on sales figures

In [17]:
cursor.execute("""
    SELECT 
        s.CurrencyCode,
        SUM(s.Quantity * p.UnitPriceUSD) AS TotalSales
    FROM sales s
    JOIN product p ON s.ProductKey = p.ProductKey
    GROUP BY s.CurrencyCode
    ORDER BY TotalSales DESC;
""")

# Fetching the results
results = cursor.fetchall()

# Creating a DataFrame
df_sales_by_currency = pd.DataFrame(results, columns=['CurrencyCode', 'TotalSales'])

# Display the first few rows of the DataFrame
df_sales_by_currency.head()


,CurrencyCode,TotalSales
0,USD,29871631.17
1,EUR,11367288.06
2,GBP,7084088.12
3,CAD,4724334.63
4,AUD,2708137.61


Store Performance: Evaluate store performance based on sales, size (square meters), and open date

In [18]:

cursor.execute("""
    SELECT 
    s.StoreKey,
    s.Country,
    s.State,
    s.SquareMeters,
    s.OpenDate,
    SUM(sa.Quantity * p.UnitPriceUSD) AS TotalSales
FROM stores s
JOIN sales sa ON s.StoreKey = sa.StoreKey
JOIN product p ON sa.ProductKey = p.ProductKey
GROUP BY s.StoreKey, s.Country, s.State, s.SquareMeters, s.OpenDate
ORDER BY TotalSales DESC
""")

# Fetching the results
results = cursor.fetchall()

# Creating a DataFrame
df_store_performance = pd.DataFrame(results, columns=['StoreKey', 'Country', 'State', 'SquareMeters', 'OpenDate', 'TotalSales'])

# Display the first few rows of the DataFrame
df_store_performance.head()


,StoreKey,Country,State,SquareMeters,OpenDate,TotalSales
0,55,United States,Nevada,2000,2009-12-15,1417885.41
1,50,United States,Kansas,2000,2008-03-06,1394738.06
2,54,United States,Nebraska,2000,2013-06-07,1384396.24
3,9,Canada,Northwest Territories,1500,2005-03-04,1336150.06
4,57,United States,New Mexico,1645,2010-06-03,1325611.89


Geographical Analysis: Analyze sales by store location to identify high-performing regions

In [19]:
cursor.execute("""
    SELECT 
        s.Country,
        s.State,
        SUM(sa.Quantity * p.UnitPriceUSD) AS TotalSales
    FROM stores s
    JOIN sales sa ON s.StoreKey = sa.StoreKey
    JOIN product p ON sa.ProductKey = p.ProductKey
    GROUP BY s.Country, s.State
    ORDER BY TotalSales DESC;
""")

# Fetching results
geographical_analysis_results = cursor.fetchall()

# Creating DataFrame for geographical analysis
df_geographical_analysis = pd.DataFrame(geographical_analysis_results, columns=['Country', 'State', 'TotalSales'])

# Display the geographical analysis results
df_geographical_analysis.head()

,Country,State,TotalSales
0,United States,Nevada,1417885.41
1,United States,Kansas,1394738.06
2,United States,Nebraska,1384396.24
3,Canada,Northwest Territories,1336150.06
4,United States,New Mexico,1325611.89
